In [1]:
!pip3 install -q pydicom

     |████████████████████████████████| 1.9MB 3.9MB/s 


In [2]:
!pip3 install -q tqdm 

In [3]:
!pip3 install -q imgaug

In [ ]:
!pip3 install pydicom kornia opencv-python scikit-image nbdev

In [5]:
import os 
import sys

import random
import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import json
import pydicom
from imgaug import augmenters as iaa
from tqdm import tqdm
import pandas as pd 
import glob
from google.colab import files
from google.colab import drive

In [6]:
import time
import datetime
from datetime import date
import os, shutil
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import itertools
import scipy
import numpy as np
import pandas as pd
import pickle
from PIL import Image
from scipy import ndimage
from sklearn.preprocessing import LabelBinarizer, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.multiclass import OneVsRestClassifier
from keras.utils.np_utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import models
from keras import layers
from keras import regularizers
from keras import optimizers
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasRegressor

np.random.seed(123)
original_start = datetime.datetime.now()
start = datetime.datetime.now()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Install Kaggle API to download competition data
# https://www.kaggle.com/general/74235

!pip3 install -q kaggle

!pip3 install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!ls

In [ ]:
files.upload()

In [ ]:
!mkdir ~/.kaggle

!cp kaggle.json ~/.kaggle/

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list

In [11]:
# enter your Kaggle credentials here
os.environ['KAGGLE_USERNAME']="mindyjen"
os.environ['KAGGLE_KEY']="d93d23dee9c18e05d21517d0fadb8efa"

In [ ]:
# directory of the project
drive.mount('/content/gdrive')
!mkdir /content/gdrive/My\ Drive/Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [13]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
# If you are unable to download the competition dataset, check to see if you have 
# accepted the user agreement on the competition website. 
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!ls
!rm -rf chest_xray/

In [ ]:
!ls

In [ ]:
!unzip \*.zip  && rm *.zip

In [19]:
# Root directory of the project
ROOT_DIR = os.path.abspath('/content/gdrive/My Drive/Kaggle')

In [ ]:
os.chdir(ROOT_DIR)
!ls

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, 'chest_xray')
os.chdir(MODEL_DIR)
!pwd
!ls

In [ ]:
# Load the images; be sure to also preprocess these into tensors 
data_pneumonia_dir = '/content/gdrive/My Drive/Kaggle/chest_xray/data/PNEUMONIA'
data_normal_dir = '/content/gdrive/My Drive/Kaggle/chest_xray/data/NORMAL'
new_dir = '/content/gdrive/My Drive/Kaggle/chest_xray/split'
DATA_DIR = os.path.join(MODEL_DIR, 'split')

if not os.path.exists(data_pneumonia_dir):
  os.makedirs(data_pneumonia_dir)

if not os.path.exists(data_normal_dir):
  os.makedirs(data_normal_dir)

if not os.path.exists(new_dir):
  os.makedirs(new_dir)

!ls

In [22]:
train_folder = os.path.join(MODEL_DIR, 'train')
train_pneumonia = os.path.join(train_folder, 'PNEUMONIA')
train_normal = os.path.join(train_folder, 'NORMAL')

test_folder = os.path.join(MODEL_DIR, 'test')
test_pneumonia = os.path.join(test_folder, 'PNEUMONIA')
test_normal = os.path.join(test_folder, 'NORMAL')

val_folder = os.path.join(MODEL_DIR, 'val')
val_pneumonia = os.path.join(val_folder, 'PNEUMONIA')
val_normal = os.path.join(val_folder, 'NORMAL')

In [ ]:
train_pneumonia

In [ ]:
imgs_train_pneumonia = [file for file in os.listdir(train_pneumonia) if file.endswith('.jpeg')]
print('There are', len(imgs_train_pneumonia), ' PNEUMONIA images for Training')

In [ ]:
imgs_test_pneumonia = [file for file in os.listdir(test_pneumonia) if file.endswith('.jpeg')]
print('There are', len(imgs_test_pneumonia), ' PNEUMONIA images for Testing')

In [ ]:
imgs_val_pneumonia = [file for file in os.listdir(val_pneumonia) if file.endswith('.jpeg')]
print('There are', len(imgs_val_pneumonia), ' PNEUMONIA images for Validating')

In [ ]:
imgs_train_normal = [file for file in os.listdir(train_normal) if file.endswith('.jpeg')]
print('There are', len(imgs_train_normal), ' NORMAL images for Training')

In [ ]:
imgs_test_normal = [file for file in os.listdir(test_normal) if file.endswith('.jpeg')]
print('There are', len(imgs_test_normal), ' NORMAL images for Testing')

In [ ]:
imgs_val_normal = [file for file in os.listdir(val_normal) if file.endswith('.jpeg')]
print('There are', len(imgs_val_normal), ' NORMAL images for Validating')

In [ ]:
!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/train/PNEUMONIA/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/PNEUMONIA
!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/test/PNEUMONIA/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/PNEUMONIA
!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/val/PNEUMONIA/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/PNEUMONIA

!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/train/NORMAL/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/NORMAL
!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/test/NORMAL/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/NORMAL
!mv /content/gdrive/My\ Drive/Kaggle/chest_xray/val/NORMAL/*.jpeg /content/gdrive/My\ Drive/Kaggle/chest_xray/data/NORMAL

In [ ]:
os.listdir(train_pneumonia)
os.listdir(train_normal)
os.listdir(test_pneumonia)
os.listdir(test_normal)
os.listdir(val_pneumonia)
os.listdir(val_normal)

In [ ]:
os.rmdir(train_pneumonia)
os.rmdir(train_normal)
os.removedirs(test_pneumonia)
os.removedirs(test_normal)
os.rmdir(val_pneumonia)
os.rmdir(val_normal)

os.chdir(MODEL_DIR)
!ls

In [ ]:
imgs_pneumonia = [file for file in os.listdir(data_pneumonia_dir) if file.endswith('.jpeg')]
print('There are', len(imgs_pneumonia), ' PNEUMONIA images')

imgs_normal = [file for file in os.listdir(data_normal_dir) if file.endswith('.jpeg')]
print('There are', len(imgs_normal), ' NORMAL images')

In [101]:
splitted_train_folder = os.path.join(DATA_DIR, 'train')
splitted_train_pneumonia = os.path.join(splitted_train_folder, 'PNEUMONIA')
splitted_train_normal = os.path.join(splitted_train_folder, 'NORMAL')

splitted_test_folder = os.path.join(DATA_DIR, 'test')
splitted_test_pneumonia = os.path.join(splitted_test_folder, 'PNEUMONIA')
splitted_test_normal = os.path.join(splitted_test_folder, 'NORMAL')

splitted_val_folder = os.path.join(DATA_DIR, 'val')
splitted_val_pneumonia = os.path.join(splitted_val_folder, 'PNEUMONIA')
splitted_val_normal = os.path.join(splitted_val_folder, 'NORMAL')

In [102]:
# training folder    
if os.path.isdir(splitted_train_folder) == False:
    os.mkdir(splitted_train_folder)
if os.path.isdir(splitted_train_pneumonia) == False:
    os.mkdir(splitted_train_pneumonia)
if os.path.isdir(splitted_train_normal) == False:
    os.mkdir(splitted_train_normal)

# testing folder
if os.path.isdir(splitted_test_folder) == False:
    os.mkdir(splitted_test_folder)
if os.path.isdir(splitted_test_pneumonia) == False:
    os.mkdir(splitted_test_pneumonia)
if os.path.isdir(splitted_test_normal) == False:
    os.mkdir(splitted_test_normal)

# validating folder
if os.path.isdir(splitted_val_folder) == False:
    os.mkdir(splitted_val_folder)
if os.path.isdir(splitted_val_pneumonia) == False:
    os.mkdir(splitted_val_pneumonia)
if os.path.isdir(splitted_val_normal) == False:  
    os.mkdir(splitted_val_normal)

In [ ]:
# training data of pneumonia
imgs = imgs_pneumonia[:3418]
for img in imgs:
    origin = os.path.join(data_pneumonia_dir, img)
    destination = os.path.join(splitted_train_pneumonia, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# training data of normal
imgs = imgs_normal[:1267]
for img in imgs:
    origin = os.path.join(data_normal_dir, img)
    destination = os.path.join(splitted_train_normal, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# testing data of pneumonia
imgs = imgs_pneumonia[3418:3845]
for img in imgs:
    origin = os.path.join(data_pneumonia_dir, img)
    destination = os.path.join(splitted_test_pneumonia, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# testing data of normal
imgs = imgs_normal[1267:1425]
for img in imgs:
    origin = os.path.join(data_normal_dir, img)
    destination = os.path.join(splitted_test_normal, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# validating data of pneumonia
imgs = imgs_pneumonia[3845:4273]
for img in imgs:
    origin = os.path.join(data_pneumonia_dir, img)
    destination = os.path.join(splitted_val_pneumonia, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# validating data of normal
imgs = imgs_normal[1425:1583]
for img in imgs:
    origin = os.path.join(data_normal_dir, img)
    destination = os.path.join(splitted_val_normal, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
imgs = imgs_normal[1425:1583]
for img in imgs:
    origin = os.path.join(data_normal_dir, img)
    destination = os.path.join(splitted_val_normal, img)
    if os.path.isfile(destination) == False:
        shutil.copyfile(origin, destination)

In [ ]:
# get all the data in the directory split/train and reshape them
print('training data:')
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    # This is the target directory
    splitted_train_folder, 
    # All images will be resized to 150x150
    target_size=(150, 150), batch_size = 4685)
    #, 
    # Since we use binary_crossentropy loss, we need binary labels
    #class_mode='binary')

In [ ]:
# get all the data in the directory split/validation and reshape them
print('validating data:')
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    splitted_val_folder, 
    target_size=(150, 150), batch_size = 792)
    #,
    #class_mode='binary')

In [ ]:
# get all the data in the directory split/test and reshape them
print('testing data:')
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
    splitted_test_folder, 
    target_size=(150, 150), batch_size = 770)
    #,
    #class_mode='binary')

In [33]:
# create the data sets
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [ ]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("*******************************************")
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("*******************************************")
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("*******************************************")
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))
print ("*******************************************")

In [ ]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

In [ ]:
print(train_labels.shape)
print(test_labels.shape)
print(val_labels.shape)

In [ ]:
train_labels

In [ ]:
train_generator.class_indices

In [ ]:
train_y = np.reshape(train_labels[:,0], (train_labels.shape[0],1))
test_y = np.reshape(test_labels[:,0], (test_labels.shape[0],1))
val_y = np.reshape(val_labels[:,0], (val_labels.shape[0],1))
print(train_y.shape)
print(test_y.shape)
print(val_y.shape)

In [40]:
# Build a baseline fully connected model
baseline = models.Sequential()
baseline.add(layers.Dense(20, activation='relu', input_shape=(67500,))) # 2 hidden layers
baseline.add(layers.Dense(7, activation='relu'))
baseline.add(layers.Dense(5, activation='relu'))
baseline.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
baseline.summary()

In [42]:
baseline.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
print('program starts, current time:')
print(start)
histoire = baseline.fit(train_img, train_y, epochs=40, batch_size=100, validation_data=(val_img, val_y))
end = datetime.datetime.now()
print('program ends, current time:')
print(end)
elapsed = end - start
print('Training took a total of {}'.format(elapsed))

In [ ]:
results_train_baseline = baseline.evaluate(train_img, train_y)

In [ ]:
results_test_baseline = baseline.evaluate(test_img, test_y)

In [ ]:
results_train_baseline

In [ ]:
results_test_baseline

In [ ]:
histoire.history.keys()

In [49]:
def visualize_training_results(results):
    history = results.history
    plt.figure()
    plt.plot(history['val_loss'])
    plt.plot(history['loss'])
    plt.legend(['val_loss', 'loss'])
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.show()
    
    plt.figure()
    plt.plot(history['val_accuracy'])
    plt.plot(history['accuracy'])
    plt.legend(['val_acc', 'acc'])
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.show()

In [ ]:
visualize_training_results(histoire)

In [51]:
preds_baseline = baseline.predict(test_img)

In [ ]:
type(preds_baseline)

In [ ]:
preds_baseline.shape

In [ ]:
test_y.shape

In [56]:
pred_baseline_class = baseline.predict_classes(test_img)

In [ ]:
pred_baseline_class.shape

In [ ]:
pred_baseline_binary_class = (baseline.predict(test_img) > 0.5).astype("int32")
pred_baseline_binary_class.shape

In [ ]:
# test set
true_classes_baseline = test_generator.classes
print(true_classes_baseline)
class_labels_baseline = list(test_generator.class_indices.keys())
class_labels_baseline

In [ ]:
accuracy_baseline = accuracy_score(test_y, preds_baseline.round())
accuracy_baseline

In [ ]:
precision_baseline = precision_score(test_y, preds_baseline.round())
precision_baseline

In [ ]:
recall_baseline = recall_score(test_y, preds_baseline.round())
recall_baseline

In [ ]:
f1_baseline = f1_score(test_y, preds_baseline.round())
f1_baseline

In [ ]:
auc_value_baseline = roc_auc_score(test_y, preds_baseline)
auc_value_baseline

In [65]:
fpr_baseline, tpr_baseline, threshold_baseline = roc_curve(test_y, preds_baseline)

In [66]:
roc_auc_baseline = auc(fpr_baseline, tpr_baseline)

In [ ]:
plt.figure()
lw=2
plt.plot(fpr_baseline, tpr_baseline, color='darkorange', lw=lw, label='ROC curve (area=%.2f)' %auc_value_baseline)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic for Baseline')
plt.legend(loc='lower right')
plt.show()

In [ ]:
report_baseline = classification_report(test_y, preds_baseline.round(), target_names=class_labels_baseline)
print(report_baseline)

In [ ]:
cnf_matrix_baseline = confusion_matrix(y_true=test_y, y_pred=preds_baseline.round())
np.set_printoptions(precision=2)
print(cnf_matrix_baseline)

In [70]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Greens):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, np.round(cm[i, j],2),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [71]:
cm_plot_labels = ['NORMAL','PNEUMONIA']

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix_baseline, classes=cm_plot_labels, title='Confusion matrix for Baseline, without normalization')
plt.show()

In [ ]:
plt.figure()
plot_confusion_matrix(cnf_matrix_baseline, classes=cm_plot_labels, normalize=True, title='Normalized confusion matrix for Baseline')
plt.show()